# Entity Resolution

##### Task: Choose the best match for each company from the suggested candidates.

## Data Exploration

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('presales_data_sample.csv')

In [10]:
pd.set_option('display.max_columns', None)
df.head(3)

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at
0,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,26e22210-93e5-11eb-b997-8dd98d09cf25,New Millennium Network,New Millennium Network Private Limited | MNET ...,New Millennium Network,PK,Pakistan,Sindh,Karachi,74400,NaN,NaN,NaN,NaN,"PK, Pakistan, Sindh, Karachi, 74400, , , 24.85...",1.0,Private,2001.0,311793.0,modelled,23.0,extracted,"MNET, incorporated in 2001, is a Pakistani com...",Mobile Top-up Services | Website Development |...,"Incorporated in 2001, MNET is the first compan...","Incorporated in 2001, MNET is the first compan...",Transfer Funds | Shop & Products | Managed Ser...,517810.0,All Other Telecommunications,NaN,NaN,Cable & Internet Providers,Telecommunication Services,IT&C Services,Media & Entertainment,SV-ME,Media,SV.1,Services,SV,7375 | 4899,Information Retrieval Services (Internet servi...,NaN,6190,Other telecommunications activities,61.9,Other telecommunications activities,NaN,9.233351e+11,+923335055787,NaN,NaN,NaN,https://www.mnet.com.pk/,mnet.com.pk,com.pk,en,NaN,NaN,NaN,http://www.linkedin.com/company/mnet-services-...,NaN,NaN,NaN,NaN,web servers: apache http server - 2 | javascri...,2020-02-25T14:47:51.000Z,2024-11-29T04:18:00.109Z
1,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,01004641-1dd8-11ef-9268-316fc8e174dd,Private Helipad Ali Villa,NaN,Private Helipad Ali Villa,PK,Pakistan,Sindh,Karachi,74400,NaN,NaN,NaN,NaN,"PK, Pakistan, Sindh, Karachi, 74400, , , 25.03...",1.0,NaN,NaN,NaN,NaN,NaN,NaN,The Private Helipad Ali Villa is a heliport lo...,Air Transportation Services | Heliport Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-29T16:24:11.019Z,2025-04-20T15:03:24.026Z
2,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,8266efc1-13e7-11ec-aa14-7bf90e1e10f1,24seven Research Network,NaN,24seven Research Network,IN,India,Haryana,Faridabad,121001,Gurunanak Gali No. 3,3071,28.399705,77.286203,"IN, India, Haryana, Faridabad, 121001, Gurunan...",1.0,Private,NaN,518659.0,modelled,75.0,modelled,24seven Research Network is a market research ...,Market Research Services | Data Collection Ser...,We are the fastest growing market research com...,"With a team of highly skilled, qualified and m...",NaN,541910.0,Marketing Research and Public Opinion Polling,NaN,NaN,Market Research & Opinion Polling,Marketing & Advertising Agencies,Advertising & Marketing,Advertising & Marketing,SV-AD,Media,SV.1,Services,SV,8732,"Commercial Economic, Sociological, and Educati...",Strategic Consultants | Information Technology...,7320,Market research and public opi

In [4]:
df.columns

Index(['input_row_key', 'input_company_name', 'input_main_country_code',
       'input_main_country', 'input_main_region', 'input_main_city',
       'input_main_postcode', 'input_main_street', 'input_main_street_number',
       'veridion_id', 'company_name', 'company_legal_names',
       'company_commercial_names', 'main_country_code', 'main_country',
       'main_region', 'main_city', 'main_postcode', 'main_street',
       'main_street_number', 'main_latitude', 'main_longitude', 'locations',
       'num_locations', 'company_type', 'year_founded', 'revenue',
       'revenue_type', 'employee_count', 'employee_count_type',
       'generated_description', 'generated_business_tags', 'short_description',
       'long_description', 'business_tags', 'naics_2022_primary_code',
       'naics_2022_primary_label', 'naics_2022_secondary_codes',
       'naics_2022_secondary_labels', 'main_business_category',
       'main_industry', 'main_sector', 'sics_codified_industry',
       'sics_codified_indu

#### Information on the dataset
The dataset contains 592 companies with 5 possible matches. The total number of records is 2951.

The input attributes are: 'input_row_key', 'input_company_name', 'input_main_country_code',
       'input_main_country', 'input_main_region', 'input_main_city',
       'input_main_postcode', 'input_main_street', 'input_main_street_number'.

In [7]:
len(df)

2951

In [8]:
len(df['input_row_key'].unique())

592

In [12]:
df.dtypes

input_row_key                int64
input_company_name          object
input_main_country_code     object
input_main_country          object
input_main_region           object
                            ...   
youtube_url                 object
tiktok_url                 float64
technologies                object
created_at                  object
last_updated_at             object
Length: 76, dtype: object

Checking if there is any differences in the country attributes

In [15]:
df['input_main_country'].unique()

array(['Pakistan', 'Denmark', 'Norway', 'Hong Kong', 'Malaysia',
       'Singapore', 'Switzerland', 'United States', 'India',
       'United Kingdom', 'Thailand', 'France', 'Ireland', 'China',
       'Finland', 'Sweden', 'Bangladesh', 'Poland', 'Germany', 'Canada',
       'South Korea', 'Luxembourg', 'Cyprus', 'Israel', 'Japan',
       'Netherlands', 'Hungary', 'Italy', 'Belgium', 'Malta',
       'United Arab Emirates', 'Serbia', 'Romania', 'Spain'], dtype=object)

In [6]:
df['main_country'].unique()

array(['Pakistan', 'India', 'Denmark', nan, 'Ireland', 'Slovakia',
       'Mauritius', 'Norway', 'Canada', 'United States', 'Hong Kong',
       'Italy', 'Malaysia', 'United Kingdom', 'Singapore', 'Mexico',
       'Switzerland', 'South Africa', 'Kenya', 'Philippines', 'Sweden',
       'Japan', 'Netherlands', 'Germany', 'Hungary', 'Thailand', 'Benin',
       'France', 'China', 'Czechia', 'Paraguay', 'Brazil',
       'United Arab Emirates', 'Latvia', 'Bulgaria', 'Costa Rica',
       'Spain', 'Turkey', 'Finland', 'Myanmar', 'Greece', 'Georgia',
       'Nigeria', 'Bangladesh', 'Ukraine', 'New Zealand', 'Australia',
       'Croatia', 'Poland', 'Saint Vincent And The Grenadines', 'Russia',
       'Austria', 'Taiwan', 'Kazakhstan', 'Honduras', 'Chile',
       'Indonesia', 'Vietnam', 'Iceland', 'Colombia', 'Estonia',
       'Argentina', 'Belgium', 'Peru', 'South Korea', 'Tanzania',
       'Luxembourg', 'Egypt', 'Democratic Republic Of The Congo',
       'Portugal', 'Vanuatu', 'Israel', 'Nepal',

## Matching the companies

In order to compare the input company name with the matching company name we will use the fuzzy matching technique. 
Therefore, each corresponding row will be assigned a score of compatibility where the highest represents a stronger match and the lowest indicates a weaker compatibility.

We will proceed with the same technique for the main city, main region and main street due to differences in terms of spelling that my occur. 

After carefully reviweing the values of country and country code we've decided to compare the 2 attributes directly against their corresponding input values using exact matches. The same applies for postal code and the street number as the format of these atributes is numerical and it should be a perfect match.

In [19]:
from rapidfuzz import fuzz

In [45]:
def matching_scores(row):
    
    scores = {}

    # Main Company name - using fuzzy matching
    scores['match_company_name'] = (
        fuzz.token_sort_ratio(row['input_company_name'], row['company_name']))

    # Main Country code - exact match
    scores['match_country_code'] = (
        1 if row['input_main_country_code'] == row['main_country_code']
        else 0
    )

    # Main Country - exact match
    scores['match_country'] = (
        1 if row['input_main_country'] == row['main_country']
        else 0
    )
    
    # Main Region- using fuzzy matching
    scores['match_region'] = (
        fuzz.token_sort_ratio(row['input_main_region'], row['main_region'])
        if pd.notnull(row['input_main_region'])
        else 0
    )

    # Main City - using fuzzy matching
    scores['match_city'] = (
        fuzz.token_sort_ratio(row['input_main_city'], row['main_city'])
        if pd.notnull(row['input_main_city'])
        else 0
    )

    # Maine Postal Code - exact match
    scores['match_postcode'] = (
        1 if pd.notnull(row['input_main_postcode']) 
        and pd.notnull(row['main_postcode'])
        and row['input_main_postcode'] == row['main_postcode'] 
        else 0
    )

    # Main Street - using fuzzy matching
    scores['match_street'] = (
        fuzz.token_sort_ratio(row['input_main_street'], row['main_street'])
        if pd.notnull(row['input_main_street']) 
        and pd.notnull(row['main_street']) 
        else 0
    )

    # Main Street Number - exact match
    scores['match_street_number'] = (
        1 if pd.notnull(row['input_main_street_number']) 
        and pd.notnull(row['main_street_number'])
        and row['input_main_street_number'] == row['main_street_number'] 
        else 0
    )

    # Based on their importance each attribute will be assigned a weight
    scores['total_score']= (
        scores['match_company_name']*10000 + # the most important - company name
        scores['match_country_code']*1000 + # second most important - country
        scores['match_country']*1000 + # second most important - country
        scores['match_region']*100 + # third most important 
        scores['match_city']*10 +
        scores['match_postcode']*10 + 
        scores['match_street']*1 +        
        scores['match_street_number']*1)

    
    return pd.Series(scores)

In [21]:
# Apply scores on the original dataframe
df_scores = df.apply(matching_scores, axis=1)

# Concat with the original dataframe
df_final = pd.concat([df, df_scores], axis=1)

We computed the compatibility scores for each input attributes and calculated a total score based on the importance of each attribute:

    -> the most important is the company name
    -> the second most important attributes are country and country code
    -> the third most important is the main reigon
    -> the fourth most important attributes are postal code and city
    -> the rest of the input attributes (street and number) are the least important as they register the most null values


In [23]:
df_final.head()

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
0,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,26e22210-93e5-11eb-b997-8dd98d09cf25,New Millennium Network,New Millennium Network Private Limited | MNET ...,New Millennium Network,PK,Pakistan,Sindh,Karachi,74400,NaN,NaN,NaN,NaN,"PK, Pakistan, Sindh, Karachi, 74400, , , 24.85...",1.0,Private,2001.0,311793.0,modelled,23.0,extracted,"MNET, incorporated in 2001, is a Pakistani com...",Mobile Top-up Services | Website Development |...,"Incorporated in 2001, MNET is the first compan...","Incorporated in 2001, MNET is the first compan...",Transfer Funds | Shop & Products | Managed Ser...,517810.0,All Other Telecommunications,NaN,NaN,Cable & Internet Providers,Telecommunication Services,IT&C Services,Media & Entertainment,SV-ME,Media,SV.1,Services,SV,7375 | 4899,Information Retrieval Services (Internet servi...,NaN,6190,Other telecommunications activities,61.9,Other telecommunications activities,NaN,9.233351e+11,+923335055787,NaN,NaN,NaN,https://www.mnet.com.pk/,mnet.com.pk,com.pk,en,NaN,NaN,NaN,http://www.linkedin.com/company/mnet-services-...,NaN,NaN,NaN,NaN,web servers: apache http server - 2 | javascri...,2020-02-25T14:47:51.000Z,2024-11-29T04:18:00.109Z,12.903226,1.0,1.0,100.000000,100.000000,0.0,0.0,0.0,142032.258065
1,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,01004641-1dd8-11ef-9268-316fc8e174dd,Private Helipad Ali Villa,NaN,Private Helipad Ali Villa,PK,Pakistan,Sindh,Karachi,74400,NaN,NaN,NaN,NaN,"PK, Pakistan, Sindh, Karachi, 74400, , , 25.03...",1.0,NaN,NaN,NaN,NaN,NaN,NaN,The Private Helipad Ali Villa is a heliport lo...,Air Transportation Services | Heliport Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-29T16:24:11.019Z,2025-04-20T15:03:24.026Z,12.307692,1.0,1.0,100.000000,100.000000,0.0,0.0,0.0,136076.923077
2,0,24-SEVEN MEDIA NETWORK (PRIVATE) LIMITED,PK,Pakistan,Sindh,Karachi,NaN,NaN,NaN,8266efc1-13e7-11ec-aa14-7bf90e1e10f1,24seven Research Network,NaN,24seven Research Network,IN,India,Haryana,Faridabad,121001,Gurunanak Gali No. 3,3071,28.399705,77.286203,"IN, India, Haryana, Faridabad, 121001, Gurunan...",1.0,Private,NaN,518659.0,modelled,75.0,modelled,24seven Research Network is a market research ...,Market Research Services | Data Collection Ser...,We are the fastest growing market research com...,"With a team of highly skilled, qualified and m...",NaN,541910.0,Marketing Research and Public Opinion Polling,NaN,NaN,Market Research & 

In [25]:
df_final['match_company_name'].describe()

count    2951.000000
mean       28.600084
std        22.138328
min         0.000000
25%        12.000000
50%        20.689655
75%        41.101056
max       100.000000
Name: match_company_name, dtype: float64

We see that more than 75% of the fuzzy compatibility scores are less than 41.

Therefore we decided that the threshold for non-acceptable matches in terms of country is 20.

In [27]:
len(df_final[df_final['total_score'] >= 200000]['input_row_key'].unique())

512

Out of the total number of 592 unique inputs, 13% don't qualify as consistent and valid. Therefore we cannot say there are reliable matches and we will exclude them from the dataset.

In [29]:
df_final = df_final[df_final['total_score'] > 200000]

In [30]:
df_final

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
6,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,92d0f2d4-0581-11ec-8c14-3f55c5871ced,CO 2 Operate,CO2 Operate B.V.,CO 2 Operate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private,NaN,NaN,NaN,15.0,modelled,"CO 2 Operate, legally known as CO2 Operate B.V...",CSR Roadmap Development | Carbon Footprint Cal...,"We are officially recognised as ""mileubaromete...",Samen bouwen aan de transitie naar een duurzam...,NaN,541620.0,Environmental Consulting Services,NaN,NaN,Environmental Engineering Services,Engineering Services,Architecture & Engineering Services,Engineering & Construction Services,IF-EN,Infrastructure,IF.2,Infrastructure,IF,8999,"Services, NEC (environmental consultants)",NaN,7490,"Other professional, scientific and technical a...",74.9 | 74.3,"Other professional, scientific and technical a...",NaN,NaN,NaN,p.burgers@co2operate.nl,NaN,NaN,https://co2operate.com/,co2operate.com,com,nl,NaN,https://twitter.com/co2operate,NaN,NaN,NaN,NaN,NaN,NaN,miscellaneous: rss | cms: wordpress - 5.9.7 | ...,2021-08-22T23:51:37.000Z,2024-11-05T09:16:37.000Z,25.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,250000.000000
8,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,0b470651-93e5-11eb-b4a6-27899be4e45d,2A Pharma ApS.,2A Pharma ApS. | 2A Pharma AB.,NaN,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,57.012738,9.988313,"DK, Denmark, North Denmark Region, Aalborg, 92...",2.0,Private,2021.0,23532880.0,modelled,35.0,modelled,2A Pharma ApS is a clinical stage biopharmaceu...,Biotechnology | Biopharmaceutical Research and...,Together we can effectively turn research and ...,Previous Next 2A Pharma is a clinical stage Sw...,Autoimmune Disease | Clinical Trials & Studies...,325414.0,Biological Product (except Diagnostic) Manufac...,NaN,NaN,Pharmaceutical Products,Pharmaceuticals,Pharmaceuticals & Cosmetics,Biotechnology & Pharmaceuticals,HC-BP,Biotechnology & Pharmaceuticals,HC.1,Health Care,HC,2836,"Biological Products, Except Diagnostic Substances",Medical Laboratories (Research) | Wholesale - ...,2100,"Manufacture of pharmaceuticals, medicinal chem...",21.2 | 21.1,Manufacture of pharmaceutical preparations | M...,808201 | 519001 | 801801 | 891001 | 591201,4.572147e+09,+4572146700,info@2apharma.com,info@2apharma.com,NaN,https://2apharma.com/,2apharma.com,com,en,NaN,https://twitter.com/2APharma,NaN,https://www.linkedin.com/company/2a-pharma-ab,NaN,NaN,NaN,NaN,widgets: instagram feed for wordpress | widget...,2020-02-17T15:49:48.000Z,2025-02-17T10:36:39.000Z,38.461538,1.0,1.0,100.000000

Since the second most important feature is the country as it is a differential aspect for each company we decided to keep only the records where the country is an exact match with the input country.

In [31]:
df_final = df_final[df_final['match_country_code'] == 1]

In [33]:
# Retrieving the best match for each input company 
perfect_matches = df_final.loc[df_final.groupby('input_row_key')['total_score'].idxmax()].reset_index(drop=True)

In [35]:
# Saving the best matches in a new file
perfect_matches.to_csv('perfect_matches.csv', index=False)

#### In conclusion, we matched the input companies with the records having the highest total score, where the most important featuers are the company name and country. 

# Data analysis

##### Task: Analyze the matched data for inconsistencies to identify what needs cleaning before client delivery.

In [37]:
perfect_matches.head()

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
0,1,2OPERATE A/S,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,0b470651-93e5-11eb-b4a6-27899be4e45d,2A Pharma ApS.,2A Pharma ApS. | 2A Pharma AB.,NaN,DK,Denmark,North Denmark Region,Aalborg,9220,Niels Jernes Vej,10,57.012738,9.988313,"DK, Denmark, North Denmark Region, Aalborg, 92...",2.0,Private,2021.0,23532880.0,modelled,35.0,modelled,2A Pharma ApS is a clinical stage biopharmaceu...,Biotechnology | Biopharmaceutical Research and...,Together we can effectively turn research and ...,Previous Next 2A Pharma is a clinical stage Sw...,Autoimmune Disease | Clinical Trials & Studies...,325414.0,Biological Product (except Diagnostic) Manufac...,NaN,NaN,Pharmaceutical Products,Pharmaceuticals,Pharmaceuticals & Cosmetics,Biotechnology & Pharmaceuticals,HC-BP,Biotechnology & Pharmaceuticals,HC.1,Health Care,HC,2836,"Biological Products, Except Diagnostic Substances",Medical Laboratories (Research) | Wholesale - ...,2100,"Manufacture of pharmaceuticals, medicinal chem...",21.2 | 21.1,Manufacture of pharmaceutical preparations | M...,808201 | 519001 | 801801 | 891001 | 591201,4.572147e+09,+4572146700,info@2apharma.com,info@2apharma.com,NaN,https://2apharma.com/,2apharma.com,com,en,NaN,https://twitter.com/2APharma,NaN,https://www.linkedin.com/company/2a-pharma-ab,NaN,NaN,NaN,NaN,widgets: instagram feed for wordpress | widget...,2020-02-17T15:49:48.000Z,2025-02-17T10:36:39.000Z,38.461538,1.0,1.0,100.000000,100.000000,1.0,100.0,1.0,397726.384615
1,2,3 STEP IT A/S,DK,Denmark,Region Of Southern Denmark,Tønder Municipality,NaN,NaN,NaN,45309031-f8ab-11ed-89fe-613148a94f23,3 Step IT A/S.,3 Step IT A/S.,NaN,DK,Denmark,Capital Region Of Denmark,Gladsaxe Municipality,2860,Vandtårnsvej,62,55.736618,12.489560,"DK, Denmark, Capital Region of Denmark, Gladsa...",1.0,Private,NaN,NaN,NaN,NaN,NaN,3 Step IT A/S is a financial institution locat...,Financial Institution | Sales Financing Services,3 Step IT A/S. is a loans & financing company ...,NaN,NaN,NaN,NaN,NaN,NaN,Loans & Financing,Banks & Financial Services,Finance & Insurance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.572118e+09,+4572118040,NaN,NaN,NaN,http://www.3stepit.dk/?opi=79508299&sa=U&ved=0...,3stepit.dk,dk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-22T14:15:04.005Z,2025-04-16T15:47:06.888Z,74.074074,1.0,1.0,66.666667,60.000000,0.0,0.0,0.0,750007.407407
2,4,3 Step IT Sweden AB,SE,Sweden,Östergötland County,Linköping,581 15,NaN,NaN,e5cede70-0409-11ed-a83d-25d09483d16d,Proact IT Sweden,Proact IT Sweden AB. | Proact Systems A/S. | P...,Proact IT Sweden,SE,Sweden,Västra Götaland Coun

In order to verify the incosistency of the data we first verified the data types of each column. In addition we checked the null values and analyzed descriptive statistics to better understand the content and distribution of the dataset.

In [38]:
perfect_matches_types = perfect_matches.applymap(type)

for col in perfect_matches.columns:
    print(f"\nColumn: {col}")
    print(perfect_matches_types[col].value_counts())
    
    null_count = perfect_matches[col].isnull().sum()
    print(f"Nulls: {null_count}")

    try:
        print("\nDescription:")
        print(perfect_matches[col].describe(include='all'))
    except Exception as e:
        print(f"Could not describe column {col}: {e}")


Column: input_row_key
<class 'int'>    449
Name: input_row_key, dtype: int64
Nulls: 0

Description:
count    449.000000
mean     296.587973
std      170.106865
min        1.000000
25%      153.000000
50%      295.000000
75%      443.000000
max      590.000000
Name: input_row_key, dtype: float64

Column: input_company_name
<class 'str'>    449
Name: input_company_name, dtype: int64
Nulls: 0

Description:
count                                          449
unique                                         448
top       APPLE DISTRIBUTION INTERNATIONAL LIMITED
freq                                             2
Name: input_company_name, dtype: object

Column: input_main_country_code
<class 'str'>    449
Name: input_main_country_code, dtype: int64
Nulls: 0

Description:
count     449
unique     28
top        DK
freq      120
Name: input_main_country_code, dtype: object

Column: input_main_country
<class 'str'>    449
Name: input_main_country, dtype: int64
Nulls: 0

Description:
count         4

We found duplicates in the input_company_name, more specifically for the company " APPLE DISTRIBUTION INTERNATIONAL LIMITED ".

Although the 2 entries have different input_row_key's the 2 records share the same attributes. Since the system returned the same matching companies and attributes for both the entries we can consider this as a duplicate.

In [39]:
perfect_matches[perfect_matches['input_company_name'] == 'APPLE DISTRIBUTION INTERNATIONAL LIMITED']

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
30,44,APPLE DISTRIBUTION INTERNATIONAL LIMITED,IE,Ireland,NaN,NaN,NaN,NaN,NaN,0086fcf1-f265-11ee-b4db-0f6fe3aa3c47,Apple green TRUCK AIR FOR WHEELS,NaN,Apple green TRUCK AIR FOR WHEELS,IE,Ireland,Leinster,Fingal,K45 FD78,Lissenhall Balbriggan Motorway,NaN,NaN,NaN,"IE, Ireland, Leinster, Fingal, K45 FD78, Lisse...",1.0,Private,NaN,NaN,NaN,NaN,NaN,Apple Green TRUCK AIR FOR WHEELS is a company ...,Transportation Services | Air Suspension Solut...,Apple green TRUCK AIR FOR WHEELS is a trucking...,NaN,NaN,484110.0,"General Freight Trucking, Local",NaN,NaN,Trucking and Logistics,Freight Transport,Transport & Warehousing,NaN,NaN,NaN,NaN,NaN,NaN,4214 | 4212,Local Trucking With Storage (general freight) ...,NaN,4923,Freight transport by road,49.41 | 49.42,Freight transport by road | Removal services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-04T09:22:25.764Z,2025-04-20T05:56:26.137Z,25.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,252000.000000
31,45,APPLE DISTRIBUTION INTERNATIONAL LIMITED,IE,Ireland,Munster,Cork,NaN,NaN,NaN,0086fcf1-f265-11ee-b4db-0f6fe3aa3c47,Apple green TRUCK AIR FOR WHEELS,NaN,Apple green TRUCK AIR FOR WHEELS,IE,Ireland,Leinster,Fingal,K45 FD78,Lissenhall Balbriggan Motorway,NaN,NaN,NaN,"IE, Ireland, Leinster, Fingal, K45 FD78, Lisse...",1.0,Private,NaN,NaN,NaN,NaN,NaN,Apple Green TRUCK AIR FOR WHEELS is a company ...,Transportation Services | Air Suspension Solut...,Apple green TRUCK AIR FOR WHEELS is a trucking...,NaN,NaN,484110.0,"General Freight Trucking, Local",NaN,NaN,Trucking and Logistics,Freight Transport,Transport & Warehousing,NaN,NaN,NaN,NaN,NaN,NaN,4214 | 4212,Local Trucking With Storage (general freight) ...,NaN,4923,Freight transport by road,49.41 | 49.42,Freight transport by road | Removal services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-04T09:22:25.764Z,2025-04-20T05:56:26.137Z,25.0,1.0,1.0,66.666667,0.0,0.0,0.0,0.0,258666.666667


The veridion_id should be unique in the perfect matches dataset.

However, we found duplicates in terms of the veridion id for the companies "ECI Media Management AB" company and "Nets Denmark A/S".
The inconsistency appeared due to varation in letter casing ( upper case and lower case) leading the system to treat them as separate records.

In [40]:
perfect_matches[perfect_matches['veridion_id'].duplicated(keep=False)]

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
30,44,APPLE DISTRIBUTION INTERNATIONAL LIMITED,IE,Ireland,NaN,NaN,NaN,NaN,NaN,0086fcf1-f265-11ee-b4db-0f6fe3aa3c47,Apple green TRUCK AIR FOR WHEELS,NaN,Apple green TRUCK AIR FOR WHEELS,IE,Ireland,Leinster,Fingal,K45 FD78,Lissenhall Balbriggan Motorway,NaN,NaN,NaN,"IE, Ireland, Leinster, Fingal, K45 FD78, Lisse...",1.0,Private,NaN,NaN,NaN,NaN,NaN,Apple Green TRUCK AIR FOR WHEELS is a company ...,Transportation Services | Air Suspension Solut...,Apple green TRUCK AIR FOR WHEELS is a trucking...,NaN,NaN,484110.0,"General Freight Trucking, Local",NaN,NaN,Trucking and Logistics,Freight Transport,Transport & Warehousing,NaN,NaN,NaN,NaN,NaN,NaN,4214 | 4212,Local Trucking With Storage (general freight) ...,NaN,4923,Freight transport by road,49.41 | 49.42,Freight transport by road | Removal services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-04T09:22:25.764Z,2025-04-20T05:56:26.137Z,25.000000,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,252000.000000
31,45,APPLE DISTRIBUTION INTERNATIONAL LIMITED,IE,Ireland,Munster,Cork,NaN,NaN,NaN,0086fcf1-f265-11ee-b4db-0f6fe3aa3c47,Apple green TRUCK AIR FOR WHEELS,NaN,Apple green TRUCK AIR FOR WHEELS,IE,Ireland,Leinster,Fingal,K45 FD78,Lissenhall Balbriggan Motorway,NaN,NaN,NaN,"IE, Ireland, Leinster, Fingal, K45 FD78, Lisse...",1.0,Private,NaN,NaN,NaN,NaN,NaN,Apple Green TRUCK AIR FOR WHEELS is a company ...,Transportation Services | Air Suspension Solut...,Apple green TRUCK AIR FOR WHEELS is a trucking...,NaN,NaN,484110.0,"General Freight Trucking, Local",NaN,NaN,Trucking and Logistics,Freight Transport,Transport & Warehousing,NaN,NaN,NaN,NaN,NaN,NaN,4214 | 4212,Local Trucking With Storage (general freight) ...,NaN,4923,Freight transport by road,49.41 | 49.42,Freight transport by road | Removal services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-04T09:22:25.764Z,2025-04-20T05:56:26.137Z,25.000000,1.0,1.0,66.666667,0.0,0.0,0.0,0.0,258666.666667
100,139,Nets Denmark A/S,DK,Denmark,Capital Region Of Denmark,Ballerup,2750,Klausdalsbrovej,601,5d5ad6b1-02d4-11ed-863c-c902831d00ca,NETS EASY MODULES,NaN,NETS EASY MODULES | EASY MODULES,DK,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"DK, Denmark, , , , , , 55.670249, 10.3333283",1.0,Private,NaN,NaN,NaN,NaN,NaN,Easy Store is an e-commerce company that opera...,Support and Guidance for Shopware | Technical ...,NETS EASY MODULES is a software development se...,NaN,E-Commerce | Smart Watches,541511.0,Custom Computer Programming Services,NaN,NaN,Software Development Services,Software & IT Services,IT&C Services,Multiline and 

In [41]:
perfect_matches[perfect_matches['company_name'] == 'ECI Media Management']

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
114,155,ECI Media Management AB,SE,Sweden,Stockholm County,Stockholm,118 53,Maria Skolgata,83,2b50edd1-93e5-11eb-a746-fdabd18be9c6,ECI Media Management,ECI Media Management AB.,ECI Media Management,SE,Sweden,Stockholm County,Stockholm,118 53,Maria Skolgata,83,59.314858,18.05413,"SE, Sweden, Stockholm County, Stockholm, 118 5...",7.0,Private,2001.0,4900000.0,extracted,82.0,extracted,"ECI Media Management, legally known as ECI Med...",Target Market Benchmarking Services | Advertis...,The benchmarking of media cost and quality per...,ECI: HIGHER MEDIA VALUE\nTechnology is transfo...,Financial Compliance | Performance Audits | Fo...,541840.0,Media Representatives,NaN,NaN,Digital & Marketing Agencies,Marketing & Advertising Agencies,Advertising & Marketing,Advertising & Marketing,SV-AD,Media,SV.1,Services,SV,7313,"Radio, Television, and Publishers’ Advertising...",Management Consultants | Marketing Consultants...,7310,Advertising,73.11 | 73.12,Advertising agencies | Media representation,641401 | 641405 | 731301 | 641403 | 641411,4.673378e+10,+46733777800,value@ecimm.com,NaN,NaN,https://ecimediamanagement.com/,ecimediamanagement.com,com,en,NaN,https://twitter.com/eci_mm,NaN,https://www.linkedin.com/company/eci-media-man...,NaN,NaN,NaN,NaN,analytics: matomo analytics | javascript frame...,2019-12-05T05:29:01.000Z,2025-01-12T13:00:05.375Z,93.023256,1.0,1.0,100.0,100.0,1.0,100.0,1.0,943343.558140
115,156,ECI MEDIA MANAGEMENT AB,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2b50edd1-93e5-11eb-a746-fdabd18be9c6,ECI Media Management,ECI Media Management AB.,ECI Media Management,SE,Sweden,Stockholm County,Stockholm,118 53,Maria Skolgata,83,59.314858,18.05413,"SE, Sweden, Stockholm County, Stockholm, 118 5...",7.0,Private,2001.0,4900000.0,extracted,82.0,extracted,"ECI Media Management, legally known as ECI Med...",Target Market Benchmarking Services | Advertis...,The benchmarking of media cost and quality per...,ECI: HIGHER MEDIA VALUE\nTechnology is transfo...,Financial Compliance | Performance Audits | Fo...,541840.0,Media Representatives,NaN,NaN,Digital & Marketing Agencies,Marketing & Advertising Agencies,Advertising & Marketing,Advertising & Marketing,SV-AD,Media,SV.1,Services,SV,7313,"Radio, Television, and Publishers’ Advertising...",Management Consultants | Marketing Consultants...,7310,Advertising,73.11 | 73.12,Advertising agencies | Media representation,641401 | 641405 | 731301 | 641403 | 641411,4.673378e+10,+46733777800,value@ecimm.com,NaN,NaN,https://ecimediamanagement.com/,ecimediamanagement.com,com,en,NaN,https://twitter.com/eci_mm,NaN,https://www.linkedin.com/company

As the maximum value for the number of employees was significantlly larger than the rest of the dataset we checked the attribute's consistency of the company "ISS GLOBAL A/S".
We confirmed that the company's number of employees is accurate with the online sources.


In [42]:
perfect_matches[perfect_matches['employee_count'] == 378946]

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
213,275,ISS GLOBAL A/S,DK,Denmark,Capital Region Of Denmark,Gladsaxe Municipality,2860,Buddingevej,197,c2b80b00-93e4-11eb-a33a-a3dfba582839,ISS,ISS A/S. | ISS GLOBAL A/S. | ISS Facility Serv...,ISS | Det Danske Rengörings Selskab | Integrat...,DK,Denmark,Capital Region Of Denmark,Søborg,NaN,NaN,NaN,NaN,NaN,"DK, Denmark, Capital Region of Denmark, Søborg...",87.0,Public,1924.0,1.004979e+10,extracted,378946.0,extracted,"ISS, legally known as ISS A/S., based in Glads...",Facility Services | Support Services | Workpla...,"Smart, integrated facility services can help y...",ISS A/S (International Service System) is a fa...,Facility Maintenance | Facilities Management,561720.0,Janitorial Services,561210.0,Facilities Support Services,Office & Industrial Cleaning,Cleaning services,Other Professional Services,Professional & Commercial Services,SV-PS,Consumer Services,SV.3,Services,SV,7342 | 4581 | 7349,Disinfecting and Pest Control Services (except...,Renovation - Commercial - low rise - 3 stories...,8121 | 8129,General cleaning of buildings | Other building...,81.29 | 81.22 | 81.21,Other cleaning activities | Other building and...,183003 | 181101 | 739401 | 739402 | 181102,9.021227e+11,+358205155 | +4538331211 | +912268511111 | +49...,malte.schaefer@iss.ch,info@de.issworld.com | press@group.issworld.co...,mandy@be.issworld.com | saskia.deiss@ch.isswor...,http://www.au.issworld.com/,issworld.com,com,en,https://www.facebook.com/issfacilityservicesda...,https://twitter.com/issworld,https://www.instagram.com/issaustria/,https://www.linkedin.com/company/iss-austria,NaN,NaN,https://www.youtube.com/user/ISSworldservicesTV,NaN,web servers: ibm websphere application server ...,2019-11-24T00:35:21.000Z,2025-04-17T18:31:54.386Z,35.294118,1.0,1.0,100.0,0.0,0.0,0.0,0.0,364941.176471


The industry, sector and business activity are consistent. The most popular in terms of entries is in the domain of Telecommunications.

We've seen that there are duplicates in the column android_app_url for 2 companies with a similar name but a different location.

In [44]:
duplicates_android = perfect_matches[perfect_matches.duplicated(subset='android_app_url', keep=False)]
duplicates_android = duplicates_android[duplicates_android['android_app_url'].notna()]
duplicates_android

,input_row_key,input_company_name,input_main_country_code,input_main_country,input_main_region,input_main_city,input_main_postcode,input_main_street,input_main_street_number,veridion_id,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city,main_postcode,main_street,main_street_number,main_latitude,main_longitude,locations,num_locations,company_type,year_founded,revenue,revenue_type,employee_count,employee_count_type,generated_description,generated_business_tags,short_description,long_description,business_tags,naics_2022_primary_code,naics_2022_primary_label,naics_2022_secondary_codes,naics_2022_secondary_labels,main_business_category,main_industry,main_sector,sics_codified_industry,sics_codified_industry_code,sics_codified_subsector,sics_codified_subsector_code,sics_codified_sector,sics_codified_sector_code,sic_codes,sic_labels,ibc_insurance_labels,isic_v4_codes,isic_v4_labels,nace_rev2_codes,nace_rev2_labels,ibc_insurance_codes,primary_phone,phone_numbers,primary_email,emails,other_emails,website_url,website_domain,website_tld,website_language_code,facebook_url,twitter_url,instagram_url,linkedin_url,ios_app_url,android_app_url,youtube_url,tiktok_url,technologies,created_at,last_updated_at,match_company_name,match_country_code,match_country,match_region,match_city,match_postcode,match_street,match_street_number,total_score
117,159,Egencia Denmark A/S,DK,Denmark,Capital Region Of Denmark,Elsinore,3000,19,19,386ddce7-93e5-11eb-b4a6-27899be4e45d,Egencia,Egencia Denmark A/S.,Egencia,DK,Denmark,Region Of Southern Denmark,Odense,5000,Kongensgade,66-68,55.398168,10.379905,"DK, Denmark, Region of Southern Denmark, Odens...",4.0,Private,NaN,NaN,NaN,NaN,NaN,"Egencia is a company based in Odense, Southern...",Travel Agency Services | Tour Operator Service...,Find løsninger til administration af erhvervsr...,Moderne administration af erhvervsrejser til e...,NaN,561510.0,Travel Agencies,NaN,NaN,Travel Agencies,Travel Agencies,Other Consumer Services,Software & IT Services,TC-SI,Technology,TC.1,Technology & Communications,TC,4724,Travel Agencies,Software Developers (Pre-Packaged) | Software ...,7911,Travel agency activities,79.11,Travel agency activities,891801 | 891802 | 740809 | 740810 | 729801,4.543327e+09,+4543327263 | +4570708000 | +4570708120 | +457...,NaN,NaN,NaN,https://www.egencia.dk/,egencia.dk,dk,da,https://www.facebook.com/amexgbt/,NaN,NaN,https://www.linkedin.com/company/american-expr...,https://apps.apple.com/us/app/egencia/id446768364,https://play.google.com/store/apps/details?id=...,https://www.youtube.com/channel/UC6RjB6tXwztw4...,NaN,javascript frameworks: requirejs | analytics: ...,2020-04-30T13:13:04.000Z,2025-04-01T00:39:00.350Z,53.846154,1.0,1.0,66.666667,28.571429,0.0,0.0,0.0,547413.919414
118,160,EGENCIA LLC,US,United States,Florida,NaN,NaN,NaN,NaN,21683381-0e12-11ec-9290-4759484bcf45,Egencia New Zealand,Egencia New Zealand Limited,Egencia New Zealand,US,United States,Illinois,Chicago,NaN,NaN,NaN,NaN,NaN,"US, United States, Washington, Seattle, , , , ...",52.0,Private,NaN,NaN,NaN,NaN,NaN,"Egencia, legally known as Egencia New Zealand ...",Industry Solutions | Ai-powered Travel and Exp...,Discover corporate travel management solutions...,Egencia is always working to deliver traveller...,Travel Manager | Business Travellers | Corpora...,561510.0,Travel Agencies,NaN,NaN,Travel Agencies,Travel Agencies,Other Consumer Services,Professional & Commercial Services,SV-PS,Consumer Services,SV.3,Services,SV,4724,Travel Agencies,Software Developers (Pre-Packaged) | Software ...,7911,Travel agency activities,79.11,Travel agency activities,891801 | 891802 | 729801 | 740809 | 740810,1.817524e+10,+18005594857 | +18175240991,nzmanagementteam@egencia.com,NaN,NaN,https://www.egencia.co.nz/,egencia.co.nz,co.nz,en,https://www.facebook.com/EgenciaBusinessTravel/,https://twitter.com/Egencia,NaN,https://www.linkedin.com/company/egencia,https://apps.apple.com/us/app/egencia/id446768364

The tiktok_url column doesnt have any entries therefore it can be excluded from the dataset.

In [55]:
perfect_matches.isnull().sum().sort_values(ascending=False).head(35)

tiktok_url                      449
android_app_url                 441
ios_app_url                     441
naics_2022_secondary_codes      423
naics_2022_secondary_labels     423
youtube_url                     400
other_emails                    389
twitter_url                     373
instagram_url                   368
emails                          311
ibc_insurance_codes             303
ibc_insurance_labels            303
facebook_url                    301
year_founded                    282
linkedin_url                    257
revenue_type                    233
revenue                         233
business_tags                   226
primary_email                   225
employee_count_type             220
employee_count                  220
technologies                    216
sics_codified_industry          213
sics_codified_sector_code       213
sics_codified_subsector_code    213
sics_codified_subsector         213
sics_codified_industry_code     213
sics_codified_sector        

More than half of the attributes contain a significant number of null values. At the top of the list, we observe that social links and domain codes are often missing. However, features like revenue, primary email, and website should not contain nulls, as they are critical for accurate matching and analysis.

# Summary & Conclusions

### Entity Resolution

In order to approach the entity resolution problem we first verified the data, its content and distribution.

After carefully analyzing we established the the 2 most important columns that can differentiate the matching rows are company name and country.
Because the company name had typos and differences in punctuation, casing and spacing we decided to use the Fuzzy Matching Technique. This method provided a similarity score between 0 and 100 where the highest represents a stronger match and the lowest indicates a weaker compatibility. The same technique was applied for the city, region and street attributes.

Regarding the country and country code we didn't found any typos or puncutation differences, therefore we decided to approach the problem differently.
We looked at the perfect matches in terms of country and since this attribute can make a difference in the accuracy of the matching company we selected only the entries that had a perfect match. Therefore, we assigned 1 for records that matched the input and 0 for those who didn't. The same technique was applied for the rest of the numerical attributes.

For each of these attributes we assigned a weight based on their importance to the final match. Company name and country were assigned a higher weight being the 2 most important valuable attributes, followed by region, city, street, postalcode. Although the last mentioned attributes don't have assigned the highest weight they can help break ties between similar records.

We then selected the entries with the highest total score that takes into consideration all the input attributes.


### Data Analysis

We verified the incosistency of the data by checking the data types of each column. In addition we checked the null values and analyzed descriptive statistics to better understand the content and distribution of the dataset.

We found duplicates in the input_company_name and veridion_id. In two duplicate cases the inconsistency appeared due to varation in letter casing ( upper case and lower case) leading the system to treat them as separate records. The matching rows were identical in terms of attributes.

The industry, sector and business activity are consistent across the dataset. The most common domain in terms of entries is Telecommunications.
Similarly, the country, region and city attributes are consistent, with Denmark being the most frequently occuring location.

We identified duplicates in the android_app_url column for two companies with similar names but different locations. Additionally, the tiktok_url column contains no entries and can therefore be excluded from the dataset.

In terms of null values, important features such as revenue and email are missing in approximately 30% of the records.
This should be addressed, as these fields are crucial for the analysis.

To conclude, the dataset presents minor incosistencies that should be analyzed in order to decide wheather to exclude or correct the affected records.